# PySpark - Pandas UDF. Скалярные типы функций (SCALAR, SCALAR_ITER) 
В данном Notebook рассматриваются типы функций: **SCALAR**, **SCALAR_ITER**

| Тип | Скалярные/Групповые | Возвращаемое значение | Трансформация |
|:----|:-----|:------|:--------------|
| **SCALAR** | Скалярные | Series | Поэлементные преобразования (скалярные вычисления) |
| **SCALAR_ITER** | Скалярные | Iterator[Series] | Батчевые преобразования (скалярные вычисления) |

### Описание  
```python
pyspark.sql.functions.pandas_udf (f=None, returnType=None, functionType=None)
#    f=None,            - Функция для преобразования в UDF
#    returnType=None,   - Тип возвращаемого значения
#    functionType=None  - Тип UDF (depricted)
```
**Тип функций:** *SCALAR*, *SCALAR_ITER*

### Параметр `returnType` (Строковые обозначения):
```python
# Примитивные типы
@pandas_udf("int")      # IntegerType
@pandas_udf("long")     # LongType  
@pandas_udf("float")    # FloatType
@pandas_udf("double")   # DoubleType
@pandas_udf("string")   # StringType
@pandas_udf("boolean")  # BooleanType
@pandas_udf("date")     # DateType
@pandas_udf("timestamp") # TimestampType

# Сложные типы
@pandas_udf("array<int>")           # ArrayType(IntegerType())
@pandas_udf("map<string,int>")      # MapType(StringType(), IntegerType())
@pandas_udf("struct<name:string,age:int>") # StructType
```

Параметр, устанавливающий максимальное количество записей в одном **Arrow batch** при передаче данных между Spark и Python процессами:
```python
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "<Количество строк в батче>")
```

In [1]:
# БД для тестовых DataSet 
DATA_DB = "pandas_udf_db"

In [2]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "python")
os.environ["SPARK_LOCAL_IP"]='localhost'
from pyspark import SparkContext, SparkConf#, HiveContext
conf = SparkConf()\
             .setAppName("Example Spark")\
             .setMaster("local[2]")\
             .setAppName("CountingSheep")\
             .set("spark.sql.catalogImplementation", "hive")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/26 01:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.7
      /_/

Using Python version 3.13.7 (main, Aug 20 2025 22:17:40)
Spark context Web UI available at http://localhost:4040
Spark context available as 'sc' (master = local[2], app id = local-1764111233941).
SparkSession available as 'spark'.


In [4]:
spark

In [5]:
from pyspark.sql.types import *
import pyspark.sql.functions as f_
from pyspark.sql import Window
from pyspark.sql import DataFrame, types
from pyspark.sql.functions import col, udf, pandas_udf, rand
from datetime import datetime, date
from pyspark.sql import Row
import pandas as pd

## Предварительная подготовка

### Тестовый DataSet

In [6]:
# Save DataSet
dfData = spark.createDataFrame([
  Row(rowid=1,  double_value1=1.75, double_value2=70.0,  str_value='Hello, World!!!',  date_value=date(2003, 6, 1), timestamp_value=datetime(2003, 1, 1, 12, 0)),
  Row(rowid=2,  double_value1=1.80, double_value2=80.0,  str_value='Python@#$%^&*()',  date_value=date(2004, 6, 2), timestamp_value=datetime(2004, 1, 2, 15, 15)),
  Row(rowid=3,  double_value1=1.65, double_value2=60.0,  str_value='Привет, мир!!! 123', date_value=date(2007, 5, 3), timestamp_value=datetime(2006, 1, 3, 19, 23)),
  Row(rowid=4,  double_value1=1.60, double_value2=60.0,  str_value=None,       date_value=date(2007, 4, 10), timestamp_value=datetime(2007, 1, 7, 20, 44)),
  Row(rowid=5,  double_value1=1.70, double_value2=90.0,  str_value='код (130) Номер 244=-55-56")', date_value=date(2008, 4, 10), timestamp_value=datetime(2008, 1, 9, 21, 34)),    
  Row(rowid=6,  double_value1=1.90, double_value2=90.0,  str_value="## Ключевые особенности 'SCALAR UDF:'", date_value=date(2009, 4, 10), timestamp_value=datetime(2009, 4, 5, 23, 53)),
  Row(rowid=7,  double_value1=1.97, double_value2=100.0, str_value="**Векто(р)и'\зованные опе%%%рации**", date_value=date(2010, 4, 10), timestamp_value=datetime(2010, 8, 23, 10, 30)), 
  Row(rowid=8,  double_value1=1.96, double_value2=110.0, str_value="?*;'';&()_",   date_value=date(2011, 4, 10), timestamp_value=datetime(2011, 2, 5, 9, 19)), 
  Row(rowid=9,  double_value1=1.83, double_value2=71.0,  str_value="Simple String №1",   date_value=date(2014, 5, 20), timestamp_value=datetime(2015, 12, 5, 19, 19)), 
  Row(rowid=10, double_value1=1.64, double_value2=87.0,  str_value="Simple String №2",   date_value=date(2015, 5, 20), timestamp_value=datetime(2016, 1, 15, 19, 10))
])

In [7]:
dfData.dtypes

[('rowid', 'bigint'),
 ('double_value1', 'double'),
 ('double_value2', 'double'),
 ('str_value', 'string'),
 ('date_value', 'date'),
 ('timestamp_value', 'timestamp')]

In [8]:
dfData.orderBy("rowid").show()

+-----+-------------+-------------+--------------------+----------+-------------------+
|rowid|double_value1|double_value2|           str_value|date_value|    timestamp_value|
+-----+-------------+-------------+--------------------+----------+-------------------+
|    1|         1.75|         70.0|     Hello, World!!!|2003-06-01|2003-01-01 12:00:00|
|    2|          1.8|         80.0|     Python@#$%^&*()|2004-06-02|2004-01-02 15:15:00|
|    3|         1.65|         60.0|  Привет, мир!!! 123|2007-05-03|2006-01-03 19:23:00|
|    4|          1.6|         60.0|                NULL|2007-04-10|2007-01-07 20:44:00|
|    5|          1.7|         90.0|код (130) Номер 2...|2008-04-10|2008-01-09 21:34:00|
|    6|          1.9|         90.0|## Ключевые особе...|2009-04-10|2009-04-05 23:53:00|
|    7|         1.97|        100.0|**Векто(р)и'\зова...|2010-04-10|2010-08-23 10:30:00|
|    8|         1.96|        110.0|          ?*;'';&()_|2011-04-10|2011-02-05 09:19:00|
|    9|         1.83|         71

In [9]:
print("Arrow batch size:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))

Arrow batch size: 10000


## PANDAS_UDF. SCALAR Example

**Особенности SCALAR UDF:**

1. **Векторизованные операции** - работает с pandas Series, а не с отдельными значениями
2. **Высокая производительность** - использует Apache Arrow для быстрой передачи данных
3. **Простота использования** - знакомый pandas API
4. **Type safety** - строгая типизация входных и выходных данных
   
**Пример определения:**
```python
from pyspark.sql.functions import PandasUDFType
# устаревший синтаксис @pandas_udf(IntegerType(), PandasUDFType.SCALAR)
@pandas_udf("int")
def slen(s):
    return s.str.len()
``` 

**SCALAR** используется, если:  
- Данные помещаются в память
- Простые преобразования без предыдущих состояний
- Не требуется инициализация дорогих ресурсов

In [10]:
import pyarrow as pa
print(f"PyArrow version: {pa.__version__}")

PyArrow version: 22.0.0


### 1. Простая функция с одним входным параметром

In [11]:
# Возведение значения в степень (квадрат)
@pandas_udf("double")
def square_value(series: pd.Series) -> pd.Series:
    """Вычисляет квадрат числа"""
    return (series ** 2).round(6)
# Применение
result = dfData.select("rowid", col("double_value1").alias("value"), square_value(col("double_value1")).alias("squared_value"))
result.orderBy("rowid").show()

[Stage 1:>                                                          (0 + 2) / 2]

+-----+-----+-------------+
|rowid|value|squared_value|
+-----+-----+-------------+
|    1| 1.75|       3.0625|
|    2|  1.8|         3.24|
|    3| 1.65|       2.7225|
|    4|  1.6|         2.56|
|    5|  1.7|         2.89|
|    6|  1.9|         3.61|
|    7| 1.97|       3.8809|
|    8| 1.96|       3.8416|
|    9| 1.83|       3.3489|
|   10| 1.64|       2.6896|
+-----+-----+-------------+



### 2. Pandas UDF с несколькими входными параметрами

In [12]:
@pandas_udf("double")
def calculate_bmi(height: pd.Series, weight: pd.Series) -> pd.Series:
    """Вычисляет индекс массы тела"""
    return (weight / (height ** 2)).round(2)

# Применение
result = dfData.select(
    "rowid", col("double_value1").alias("height"), col("double_value2").alias("weight"),
    calculate_bmi(col("height"), col("weight")).alias("bmi")
)
result.orderBy("rowid").show()

+-----+------+------+-----+
|rowid|height|weight|  bmi|
+-----+------+------+-----+
|    1|  1.75|  70.0|22.86|
|    2|   1.8|  80.0|24.69|
|    3|  1.65|  60.0|22.04|
|    4|   1.6|  60.0|23.44|
|    5|   1.7|  90.0|31.14|
|    6|   1.9|  90.0|24.93|
|    7|  1.97| 100.0|25.77|
|    8|  1.96| 110.0|28.63|
|    9|  1.83|  71.0| 21.2|
|   10|  1.64|  87.0|32.35|
+-----+------+------+-----+



### 3. UDF со строковыми преобразованиями

In [13]:
import re

# Тестовые данные с "грязным" текстом
@pandas_udf("string")
def clean_text(text_series: pd.Series) -> pd.Series:
    """Очистка текста от специальных символов"""
    def clean_string(text):
        if text is None:
            return None
        # Удаляем все кроме букв, цифр и пробелов
        return re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', str(text)).strip()
    return text_series.apply(clean_string)

result = dfData.select("rowid", "str_value", clean_text(col("str_value")).alias("cleaned_value"))
result.orderBy("rowid").show(truncate=False)

+-----+-------------------------------------+-------------------------------+
|rowid|str_value                            |cleaned_value                  |
+-----+-------------------------------------+-------------------------------+
|1    |Hello, World!!!                      |Hello World                    |
|2    |Python@#$%^&*()                      |Python                         |
|3    |Привет, мир!!! 123                   |Привет мир 123                 |
|4    |NULL                                 |NULL                           |
|5    |код (130) Номер 244=-55-56")         |код 130 Номер 2445556          |
|6    |## Ключевые особенности 'SCALAR UDF:'|Ключевые особенности SCALAR UDF|
|7    |**Векто(р)и'\зованные опе%%%рации**  |Векторизованные операции       |
|8    |?*;'';&()_                           |                               |
|9    |Simple String №1                     |Simple String 1                |
|10   |Simple String №2                     |Simple String 2    

### 4. Математические вычисления

**Сложные математические вычисления** - сложная нелинейная функция
```
f(x) = log(x + 1) × √x + sin(x)

где:
ln — натуральный логарифм (основание e)
√x — квадратный корень от x
sin(x) — синус от x (в радианах)
```

**Z-score нормализация** — метод масштабирования данных, который преобразует значения так, чтобы они имели среднее значение 0 и стандартное отклонение 1.
```
Z = (X - μ) / σ

где:
Z = нормализованное значение (z-score)
X = исходное значение
μ = среднее значение выборки
σ = стандартное отклонение выборки
```

In [14]:
import numpy as np

@pandas_udf("double")
def complex_calculation(series: pd.Series) -> pd.Series:
    """Сложные математические вычисления"""
    return np.log(series + 1) * np.sqrt(series) + np.sin(series)

@pandas_udf("double")
def normalize_zscore(series: pd.Series) -> pd.Series:
    """Z-score нормализация"""
    mean_val = series.mean()
    std_val = series.std()
    return (series - mean_val) / std_val

# Применение
data = [(i, float(i * 10 + np.random.randn())) for i in range(1, 11)]
dfComplex = spark.createDataFrame(data, ["rowid", "value"])

result = dfComplex.select(
    "rowid", "value",
    complex_calculation(col("value")).alias("complex_result"),
    normalize_zscore(col("value")).alias("normalized")
)
result.show()

+-----+------------------+------------------+--------------------+
|rowid|             value|    complex_result|          normalized|
+-----+------------------+------------------+--------------------+
|    1|10.663677245877121| 7.076278235298124| -1.2240494902983483|
|    2| 19.60486931373665|14.081761492402679| -0.6708086298707555|
|    3| 29.66178526445094| 17.65941347704602|-0.04853178002861...|
|    4| 41.56500955006113|23.520640946921596|  0.6879863481643397|
|    5| 50.73531233369813|  28.5605906769476|  1.2554035520333788|
|    6|  60.3544999140733|31.365179174605387| -1.2355627443590032|
|    7| 68.84843359065881| 34.97042746837918|  -0.683202185499726|
|    8| 79.73964603043659| 38.28041232910466|0.025053425969692986|
|    9| 88.89726635828859| 43.21905741184908|  0.6205735318138446|
|   10|  98.9320868869754|44.798782779304176|   1.273137972075187|
+-----+------------------+------------------+--------------------+



### 5. Обработка дат

In [15]:
from datetime import datetime

@pandas_udf("string")
def format_date(date_series: pd.Series) -> pd.Series:
    """Форматирование даты"""
    return pd.to_datetime(date_series).dt.strftime('%Y-%m-%d %A')

@pandas_udf("int")
def days_since_epoch(date_series: pd.Series) -> pd.Series:
    """Количество дней с начала эпохи"""
    epoch = pd.Timestamp('1970-01-01')
    return (pd.to_datetime(date_series) - epoch).dt.days

# Тестовые данные
from datetime import date, timedelta
dates_data = [
    (1, date.today()),
    (2, date.today() - timedelta(days=30)),
    (3, date.today() + timedelta(days=15))
]

result = dfData.select(
    "rowid", "date_value",
    format_date(col("date_value")).alias("formatted_date"),
    days_since_epoch(col("date_value")).alias("days_since_epoch")
)
result.orderBy("rowid").show(truncate=False)

+-----+----------+--------------------+----------------+
|rowid|date_value|formatted_date      |days_since_epoch|
+-----+----------+--------------------+----------------+
|1    |2003-06-01|2003-06-01 Sunday   |12204           |
|2    |2004-06-02|2004-06-02 Wednesday|12571           |
|3    |2007-05-03|2007-05-03 Thursday |13636           |
|4    |2007-04-10|2007-04-10 Tuesday  |13613           |
|5    |2008-04-10|2008-04-10 Thursday |13979           |
|6    |2009-04-10|2009-04-10 Friday   |14344           |
|7    |2010-04-10|2010-04-10 Saturday |14709           |
|8    |2011-04-10|2011-04-10 Sunday   |15074           |
|9    |2014-05-20|2014-05-20 Tuesday  |16210           |
|10   |2015-05-20|2015-05-20 Wednesday|16575           |
+-----+----------+--------------------+----------------+



### 6. Условная логика

In [16]:
@pandas_udf("string")
def categorize_value(series: pd.Series) -> pd.Series:
    """Категоризация значений"""
    def categorize(value):
        if value < 10:
            return "Low"
        elif value < 50:
            return "Medium"
        else:
            return "High"
    
    return series.apply(categorize)

@pandas_udf("boolean")
def is_outlier(series: pd.Series) -> pd.Series:
    """Определение выбросов (значения вне 2 стандартных отклонений)"""
    mean_val = series.mean()
    std_val = series.std()
    return (series - mean_val).abs() > 2 * std_val

# Применение
data = [(i, float(i * 5 + np.random.randn() * 10)) for i in range(1, 21)]
dfLogic = spark.createDataFrame(data, ["id", "value"])

result = dfLogic.select(
                "id", "value",
                 categorize_value(col("value")).alias("category"),
                 is_outlier(col("value")).alias("is_outlier")
                        )

In [17]:
result.show()

+---+------------------+--------+----------+
| id|             value|category|is_outlier|
+---+------------------+--------+----------+
|  1|14.598566930062413|  Medium|     false|
|  2| 18.09290069910137|  Medium|     false|
|  3| 9.046838823206393|     Low|     false|
|  4|20.161580902334457|  Medium|     false|
|  5|28.221032477005032|  Medium|     false|
|  6|39.477603580964114|  Medium|     false|
|  7| 44.25479347454431|  Medium|     false|
|  8| 49.17795107489015|  Medium|     false|
|  9| 56.57539048925414|    High|     false|
| 10|47.726490648508225|  Medium|     false|
| 11| 60.25029361948212|    High|     false|
| 12| 68.22748336391605|    High|     false|
| 13| 47.33976697164026|  Medium|     false|
| 14|59.307177481097376|    High|     false|
| 15| 81.34310309234621|    High|     false|
| 16| 85.16523399004258|    High|     false|
| 17| 63.27064366793742|    High|     false|
| 18|  95.9583632678565|    High|     false|
| 19| 89.72266209231665|    High|     false|
| 20|100.6

### 7. Сравнение производительности UDF - Pandas UDF
Произодится генерация 500000 строк, к которым применяется последовательно UDF и Pandas UDF, содержащие идентичное преобразование (возведение числа в квадрат )

In [18]:
# Обычная UDF (медленная)
@udf(returnType=DoubleType())
def slow_square(value):
    return float(value ** 2)

# Pandas UDF (быстрая)
@pandas_udf("double")
def fast_square(series: pd.Series) -> pd.Series:
    return series ** 2

# Большой датасет для тестирования
large_data = [(i, float(i)) for i in range(500000)]
large_df = spark.createDataFrame(large_data, ["id", "value"])

**Сравнение производительности**

In [19]:
import time

# Pandas UDF
start = time.time()
result_pandas = large_df.select("id", fast_square(col("value")).alias("squared"))
result_pandas.count()  # Trigger execution
pandas_time = time.time() - start
print(f"Pandas UDF time: {pandas_time:.2f} seconds")

result_pandas = large_df.select("id", slow_square(col("value")).alias("squared"))
result_pandas.count()  # Trigger execution
pandas_time = time.time() - start
print(f"UDF time: {pandas_time:.2f} seconds")


Pandas UDF time: 0.46 seconds
UDF time: 0.67 seconds


## PANDAS_UDF.SCALAR_ITER Example

**Особенности SCALAR_ITER UDF (в дополнение к особенностям SCALAR):**

1. **Батчевая обработка данных** - Данные поступают **частями**, а не все сразу
2. **Стабильность** - Размер батча контролируется Spark (обычно тысячи записей)
3. **Масштабируемость** - Позволяет обрабатывать **очень большие датасеты**
4. **Управление памятью** - В памяти одновременно только один батч
5. **Накопительные вычисления** - Использует накопленную информацию (возможность кэширования)
6. **Потоковые агрегации** - Можно производить вычисления по окну
7. **Обработка ошибок** -  Можно обрабатывать ошибки на уровне батчей

**Преимущества SCALAR_ITER:**

- **Контроль памяти** - обрабатывает данные по частям
- **Эффективность** - можно переиспользовать дорогие объекты
- **Гибкость** - позволяет сохранять состояние между батчами

SCALAR_ITER особенно полезен для обработки больших объемов данных с ограниченной памятью

**Тестовые данные**
Создается DataFrame 4000 строк - 5 партиций (~800 строк)

In [20]:
sampleData = [(i, float(i)) for i in range(1, 4001)]
dfSample = spark.createDataFrame(sampleData, ["rowid", "value"]).repartition(5)

In [21]:
print(f"Размеры partitions: {dfSample.rdd.glom().map(len).collect()}")

Размеры partitions: [801, 800, 799, 799, 801]


**Установить размер батча**  
```spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "250")```

✅ **Увеличить batch_size если используются:**
- Простые операции (арифметические, простые функции)
- Много доступной памяти
- Нужна максимальная производительность
- Большие датасеты

⬇️ **Уменьшить batch_size если используются:**
- Сложные операции (ML, тяжелые вычисления)
- Ограниченная память
- Нестабильные данные (много ошибок)
- Потоковая обработка

In [22]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "650")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

### 1. Базовый пример SCALAR_ITER
Представлен общий принцип организации **pandas_udf** с батчевой обработкой.  
 - Создаются две функции: **SCALAR** и **SCALAR_ITER**  
 - Последовательно обрабатывается DataFrame **dfSample**

In [23]:
from typing import Iterator

# SCALAR - работает с отдельными Series
@pandas_udf("double")
def scalar_udf(series: pd.Series) -> pd.Series:
    print(f"SCALAR.Обрабатывается series {len(series)} строк")
    return series ** 2

# SCALAR_ITER Pandas UDF с вызовом итератора в теле функции
@pandas_udf("double")
def scalar_iter_udf(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Обрабатывает данные батчами для экономии памяти"""
    batch_count = 0
    total_rows = 0
    for  i, batch in enumerate(iterator):
        # Обработка каждого батча
        total_rows += len(batch)
        batch_count += 1
        yield batch ** 2
    print(f"SCALAR_ITER.Обрабатывается партиция {total_rows} строк в {batch_count} батчах")        

✅ **SCALAR UDF** обрабатывает series с максимальным количеством строк, равным **maxRecordsPerBatch**  
(или размеру партиции в случае, если количество строк в партиции меньше maxRecordsPerBatch)

In [24]:
%%time
result = dfSample.select("rowid", scalar_udf(col("value")).alias("squared"))
ret = result.collect()

CPU times: user 9.88 ms, sys: 996 μs, total: 10.9 ms
Wall time: 303 ms


SCALAR.Обрабатывается series 650 строкSCALAR.Обрабатывается series 650 строк

SCALAR.Обрабатывается series 151 строкSCALAR.Обрабатывается series 150 строк

SCALAR.Обрабатывается series 650 строк
SCALAR.Обрабатывается series 149 строк
SCALAR.Обрабатывается series 650 строк
SCALAR.Обрабатывается series 149 строк
SCALAR.Обрабатывается series 650 строк
SCALAR.Обрабатывается series 151 строк


✅ **SCALAR_ITER UDF** обрабатывает каждую партицию батчами максимальным количеством строк, равным **maxRecordsPerBatch**

In [25]:
%%time
result = dfSample.select("rowid", scalar_iter_udf(col("value")).alias("squared"))
ret = result.collect()

CPU times: user 11.4 ms, sys: 1.93 ms, total: 13.3 ms
Wall time: 278 ms


SCALAR_ITER.Обрабатывается партиция 800 строк в 2 батчах
SCALAR_ITER.Обрабатывается партиция 801 строк в 2 батчах
SCALAR_ITER.Обрабатывается партиция 799 строк в 2 батчах
SCALAR_ITER.Обрабатывается партиция 799 строк в 2 батчах
SCALAR_ITER.Обрабатывается партиция 801 строк в 2 батчах


### 2. Обработка данных с состоянием между батчами
**Состояние между батчами** — это возможность сохранять и накапливать данные при обработке нескольких батчей данных. Позволяет создавать сложные алгоритмы потоковой обработки, которые "помнят" информацию из предыдущих порций данных.

**Основной принцип:**
```python
from typing import Iterator
import pandas as pd

@pandas_udf("double")
def stateful_processing(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
   # Состояние инициализируется один раз для всей partition
   state_value = 0  # Состояние сохраняется между батчами
    
   for batch in iterator:  # Каждый batch обрабатывается последовательно
       # Обновляется состояние переменной
       state_value = <какой-то код>
        
       # Результат с учетом состояния
       yield batch + state_value
```

**Состояние между батчами** 

✅ **Сохраняется:**
- Переменные, объявленные вне цикла `for batch in iterator`
- Коллекции (списки, множества, словари)
- Счетчики и аккумуляторы
- Буферы для окон данных

❌ **НЕ сохраняется:**
- Состояние между разными partition
- Состояние между разными executor'ами
- Переменные внутри цикла батчей

**Сравнение со SCALAR UDF:**
```python
# ❌ SCALAR - состояние НЕ сохраняется
@pandas_udf("double")
def scalar_no_state(series: pd.Series) -> pd.Series:
    counter = 0  # Сбрасывается при каждом вызове
    return series.cumsum()  # Работает только внутри series

# ✅ SCALAR_ITER - состояние сохраняется
@pandas_udf("double")
def scalar_iter_with_state(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    counter = 0  # Сохраняется между батчами
    
    for batch in iterator:
        counter += len(batch)
        yield batch + counter  # Учитывает все предыдущие батчи
```

#### **Пример расчета статистики с накоплением между батчами**
Обрабатывается batch, размером 10 строк. Рассчитывается среднее арифметическое значение.  
 - для 1-го батча - расчет только для него (и запоминает значение)
 - для 2-го батча - расчет для 1-го и 2-го совокупно
 - для 3-го батча - расчет для 1-го, 2-го ... по N-ый совокупно

⚠️ **Функция обменивается данными только внутри батчей одной партиции**

In [26]:
# Настройки
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "10")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [27]:
@pandas_udf("double")
def partitiom_statistics(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Вычисляет статистики с накоплением между батчами"""
    running_sum = 0
    running_count = 0
    
    for batch in iterator:
        running_sum += batch.sum()
        running_count += len(batch)
        print(f"Обрабатывается батч размером: {len(batch)}")                    
        print(f"running_sum = {running_sum}, running_count = {running_count}")
        # Возвращаем среднее значение на текущий момент
        current_mean = running_sum / running_count
        yield pd.Series([current_mean] * len(batch))

In [28]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", dfSample.rdd.getNumPartitions())
print("Размеры partitions:", dfSample.rdd.glom().map(len).collect())

Размер батча Arrow: 10
Количество partitions: 5
Размеры partitions: [801, 800, 799, 799, 801]


In [29]:
# Применение для 11 строк - вывод для одного батча
result = dfSample.select("rowid", col("value"), partitiom_statistics(col("value")).alias("running_mean"))
result.show(10)

+-----+------+------------+
|rowid| value|running_mean|
+-----+------+------------+
| 1728|1728.0|       918.7|
|  174| 174.0|       918.7|
|  138| 138.0|       918.7|
| 1758|1758.0|       918.7|
| 1719|1719.0|       918.7|
| 1310|1310.0|       918.7|
|  202| 202.0|       918.7|
| 1375|1375.0|       918.7|
|  395| 395.0|       918.7|
|  388| 388.0|       918.7|
+-----+------+------------+
only showing top 10 rows



Обрабатывается батч размером: 10
running_sum = 9187.0, running_count = 10
Обрабатывается батч размером: 1
running_sum = 10927.0, running_count = 11


In [30]:
# Применение для 21 строки - вывод для двух батчей
result = dfSample.select("rowid", col("value"), partitiom_statistics(col("value")).alias("running_mean"))
result.show(20)

+-----+------+------------+
|rowid| value|running_mean|
+-----+------+------------+
| 1728|1728.0|       918.7|
|  174| 174.0|       918.7|
|  138| 138.0|       918.7|
| 1758|1758.0|       918.7|
| 1719|1719.0|       918.7|
| 1310|1310.0|       918.7|
|  202| 202.0|       918.7|
| 1375|1375.0|       918.7|
|  395| 395.0|       918.7|
|  388| 388.0|       918.7|
| 1740|1740.0|      917.65|
| 1369|1369.0|      917.65|
|  582| 582.0|      917.65|
|  298| 298.0|      917.65|
|  920| 920.0|      917.65|
|  641| 641.0|      917.65|
| 1054|1054.0|      917.65|
|  291| 291.0|      917.65|
|  406| 406.0|      917.65|
| 1865|1865.0|      917.65|
+-----+------+------------+
only showing top 20 rows



Обрабатывается батч размером: 10
running_sum = 9187.0, running_count = 10
Обрабатывается батч размером: 10
running_sum = 18353.0, running_count = 20
Обрабатывается батч размером: 1
running_sum = 19347.0, running_count = 21


#### **Пример расчета накопительной суммы (Cumulative Sum)** и **Скользящего среднего** (между батчами)
Еще один пример расчета с учетом состояний между батчами  

⚠️ **Функция обменивается данными только внутри батчей одной партиции**

In [31]:
# Создается тестовый DataFrame 5 партиций, размер батча: 10 
data = [(i, float(i)) for i in range(1, 31)]
df_example_2 = spark.createDataFrame(data, ["rowid", "value"]).repartition(5)

In [32]:
from collections import deque

@pandas_udf("double")
def cross_batch_cumulative_sum(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Накопительную сумму через батчи"""
    running_total = 0.0
    
    for batch in iterator:
        batch_cumsum = batch.cumsum()            # Вычисляется cumsum внутри батча
        result = batch_cumsum + running_total    # Накопленное значение с предыдущих батчей
        # Обновление состояни/ для следующего батча
        running_total += batch.sum()
        yield result

@pandas_udf("double")
def cross_batch_moving_avg(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Скользящее среднее с окном, пересекающим батчи"""
    window_size = 5
    window_buffer = deque(maxlen=window_size)
    
    for batch in iterator:
        results = []
        
        for value in batch:
            window_buffer.append(value)
            current_avg = sum(window_buffer) / len(window_buffer) # Вычисляется среднее по текущему окну
            results.append(current_avg)
        
        yield pd.Series(results)

In [33]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", df_example_2.rdd.getNumPartitions())
print("Размеры partitions:", df_example_2.rdd.glom().map(len).collect())

Размер батча Arrow: 10
Количество partitions: 5
Размеры partitions: [6, 6, 6, 6, 6]


**Применение на 10 батчах в каждой из 5 партиций**

In [34]:
# Применение r
df_example_2.select("rowid", "value"
                    ,cross_batch_cumulative_sum(col("value")).alias("cumulative_sum")
                    ,cross_batch_moving_avg(col("value")).alias("moving_avg")
                    ).show()

+-----+-----+--------------+-----------------+
|rowid|value|cumulative_sum|       moving_avg|
+-----+-----+--------------+-----------------+
|   11| 11.0|          11.0|             11.0|
|    9|  9.0|          20.0|             10.0|
|    8|  8.0|          28.0|9.333333333333334|
|   28| 28.0|          56.0|             14.0|
|   19| 19.0|          75.0|             15.0|
|   25| 25.0|         100.0|             17.8|
|    6|  6.0|           6.0|              6.0|
|   15| 15.0|          21.0|             10.5|
|   12| 12.0|          33.0|             11.0|
|   30| 30.0|          63.0|            15.75|
|   16| 16.0|          79.0|             15.8|
|   29| 29.0|         108.0|             20.4|
|    3|  3.0|           3.0|              3.0|
|    4|  4.0|           7.0|              3.5|
|    7|  7.0|          14.0|4.666666666666667|
|   22| 22.0|          36.0|              9.0|
|   27| 27.0|          63.0|             12.6|
|   24| 24.0|          87.0|             16.8|
|    1|  1.0|

**Применение на 10 батчах для одной партиции** (сортировка приводит DataFrame к одной партиции)

In [35]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", df_example_2.orderBy("rowid").rdd.getNumPartitions())
print("Размеры partitions:", df_example_2.orderBy("rowid").rdd.glom().map(len).collect())

Размер батча Arrow: 10
Количество partitions: 1
Размеры partitions: [30]


In [36]:
# Применение c сортировкой
df_example_2.orderBy("rowid")\
            .select("rowid", "value"
                    ,cross_batch_cumulative_sum(col("value")).alias("cumulative_sum")
                    ,cross_batch_moving_avg(col("value")).alias("moving_avg")
                   ).show()

+-----+-----+--------------+----------+
|rowid|value|cumulative_sum|moving_avg|
+-----+-----+--------------+----------+
|    1|  1.0|           1.0|       1.0|
|    2|  2.0|           3.0|       1.5|
|    3|  3.0|           6.0|       2.0|
|    4|  4.0|          10.0|       2.5|
|    5|  5.0|          15.0|       3.0|
|    6|  6.0|          21.0|       4.0|
|    7|  7.0|          28.0|       5.0|
|    8|  8.0|          36.0|       6.0|
|    9|  9.0|          45.0|       7.0|
|   10| 10.0|          55.0|       8.0|
|   11| 11.0|          66.0|       9.0|
|   12| 12.0|          78.0|      10.0|
|   13| 13.0|          91.0|      11.0|
|   14| 14.0|         105.0|      12.0|
|   15| 15.0|         120.0|      13.0|
|   16| 16.0|         136.0|      14.0|
|   17| 17.0|         153.0|      15.0|
|   18| 18.0|         171.0|      16.0|
|   19| 19.0|         190.0|      17.0|
|   20| 20.0|         210.0|      18.0|
+-----+-----+--------------+----------+
only showing top 20 rows



#### **Пример реализацмм счетчика уникальных значений**

In [37]:
# Данные с повторениями
data = [(1, "A"), (2, "B"), (3, "A"), (4, "C"), (5, "B"), (6, "D"), (7, "E"), (8, "F")]
df_unique_count_example = spark.createDataFrame(data, ["rowid", "value"])

In [38]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", df_unique_count_example.rdd.getNumPartitions())
print("Размеры partitions:", df_unique_count_example.rdd.glom().map(len).collect())

Размер батча Arrow: 10
Количество partitions: 2
Размеры partitions: [4, 4]


In [39]:
@pandas_udf("long")
def unique_count_stateful(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Подсчет уникальных значений с состоянием"""
    seen_values = set()  # Состояние: множество уже провереных значений
    
    for batch in iterator:
        results = []
        
        for value in batch:
            seen_values.add(value)
            # Возвращаем текущее количество уникальных значений
            results.append(len(seen_values))
        
        yield pd.Series(results)

In [40]:
result = df_unique_count_example.select("rowid", "value", 
                                         unique_count_stateful(col("value")).alias("unique_count"))
result.show()

+-----+-----+------------+
|rowid|value|unique_count|
+-----+-----+------------+
|    1|    A|           1|
|    2|    B|           2|
|    3|    A|           2|
|    4|    C|           3|
|    5|    B|           1|
|    6|    D|           2|
|    7|    E|           3|
|    8|    F|           4|
+-----+-----+------------+



#### **Пример реализацмм обнаружения аномалий с адаптивным порогом**

In [41]:
# DataSet с аномалиями
np.random.seed(42)
normal_data = list(np.random.normal(50, 10, 18))
anomaly_data = [150, 200]  # Аномалии
all_data = normal_data + anomaly_data

data = [(i+1, float(val)) for i, val in enumerate(all_data)]
dfAnomalyDate = spark.createDataFrame(data, ["rowid", "value"])

In [42]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", dfAnomalyDate.rdd.getNumPartitions())
print("Размеры partitions:", dfAnomalyDate.rdd.glom().map(len).collect())

Размер батча Arrow: 10
Количество partitions: 2
Размеры partitions: [10, 10]


In [43]:
import numpy as np

@pandas_udf("boolean")
def adaptive_anomaly_detection(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Обнаружение аномалий с адаптивным порогом"""
    # Состояние: статистики для адаптации порога
    running_mean = 0.0
    running_var = 0.0
    count = 0
    
    for batch in iterator:
        results = []
        
        for value in batch:
            count += 1
            
            # Обновляем running statistics (Welford's algorithm)
            delta = value - running_mean
            running_mean += delta / count
            delta2 = value - running_mean
            running_var += delta * delta2
            
            # Вычисляем текущее стандартное отклонение
            if count > 1:
                current_std = np.sqrt(running_var / (count - 1))
                threshold = 2 * current_std                
                # Проверяем на аномалию
                is_anomaly = abs(value - running_mean) > threshold
            else:
                is_anomaly = False
            
            results.append(is_anomaly)
        
        yield pd.Series(results)

In [44]:
result = dfAnomalyDate.select("rowid", "value", 
                  adaptive_anomaly_detection(col("value")).alias("is_anomaly"))
result.show()

+-----+------------------+----------+
|rowid|             value|is_anomaly|
+-----+------------------+----------+
|    1| 54.96714153011233|     false|
|    2| 48.61735698828815|     false|
|    3| 56.47688538100692|     false|
|    4| 65.23029856408026|     false|
|    5|47.658466252766644|     false|
|    6|  47.6586304305082|     false|
|    7| 65.79212815507391|     false|
|    8| 57.67434729152909|     false|
|    9| 45.30525614065048|     false|
|   10| 55.42560043585965|     false|
|   11|45.365823071875376|     false|
|   12| 45.34270246429743|     false|
|   13| 52.41962271566034|     false|
|   14| 30.86719755342202|     false|
|   15| 32.75082167486967|     false|
|   16| 44.37712470759027|     false|
|   17| 39.87168879665576|     false|
|   18|53.142473325952736|     false|
|   19|             150.0|      true|
|   20|             200.0|      true|
+-----+------------------+----------+



### 3. **Обработка с кэшированием**
Мощный механизм оптимизации обработки данных с помошью **pandas_udf** (SCALAR_ITER)  
**Основной принцип:**
Сохранение расчитанных результатов в **cache** (ключ-знчение) , для избегания повтрных расчетов
```python
from typing import Iterator
import pandas as pd

@pandas_udf("string")
def processing_with_cache(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    # Кэш для избежания повторных вычислений
    cache = {}
    
    def processing_code(value):
        if value in cache:
            return cache[value]
        
        if value is None:
            result = None
        else:
            result =  <Сложная обработка>
        
        cache[value] = result
        return result
    
    for batch in iterator:
        processed_batch = batch.apply(processing_code)
        yield processed_batch
```

#### **Простой пример с кэшироанием**

In [45]:
# Данные с повторениями для демонстрации кэширования
text_data = [
    (1, "APACHE"), (2, "HIVE"), (3, "APACHE"),  # APACHE повторяется
    (4, "PYTHON"), (5, "HIVE"), (6, "SPARK")    # HIVE повторяется
]
df_text = spark.createDataFrame(text_data, ["id", "text"])

In [46]:
# Настройки
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "3")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [47]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", df_text.rdd.getNumPartitions())
print("Размеры partitions:", df_text.rdd.glom().map(len).collect())

Размер батча Arrow: 3
Количество partitions: 2
Размеры partitions: [3, 3]


In [48]:
@pandas_udf("string")
def incremental_text_processing(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Инкрементальная обработка текста с кэшем"""
    # Состояние: кэш обработанных значений
    processing_cache = {}
    processing_count = 0
    
    def expensive_text_operation(text):
        """Имитация дорогостоящей операции обработки текста"""
        return text.upper().replace('A', '@').replace('E', '3')
    
    for batch in iterator:
        results = []
        
        for text in batch:
            if text in processing_cache:
                # Кэшированный результат
                result = processing_cache[text]
            else:
                # Выполнение дорогой операции
                result = expensive_text_operation(text)
                processing_cache[text] = result
                processing_count += 1
        
            results.append(result)
        
        yield pd.Series(results)

In [49]:
result = df_text.select("id", "text", 
                       incremental_text_processing(col("text")).alias("processed"))
result.show()

+---+------+---------+
| id|  text|processed|
+---+------+---------+
|  1|APACHE|   @P@CH3|
|  2|  HIVE|     HIV3|
|  3|APACHE|   @P@CH3|
|  4|PYTHON|   PYTHON|
|  5|  HIVE|     HIV3|
|  6| SPARK|    SP@RK|
+---+------+---------+



#### **Пример с кэшированием для операции с регулярными выражениями**

In [50]:
import re
from collections import defaultdict

@pandas_udf("string")
def text_processing_with_cache(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Обработка текста с кэшированием результатов"""
    
    # Кэш для избежания повторных вычислений
    cache = {}
    
    def clean_text(text):
        if text in cache:
            return cache[text]
        
        if text is None:
            result = None
        else:
            # Сложная обработка текста r'[^a-zA-Z0-9\s]'
            result = re.sub(r'[^a-zA-Z0-9а-яА-ЯёЁ\s]', '', str(text)).strip().upper()

        cache[text] = result
        return result
    
    for batch in iterator:
        processed_batch = batch.apply(clean_text)
        yield processed_batch

In [51]:
result = dfData.select("rowid", "str_value", text_processing_with_cache(col("str_value")).alias("cleaned"))
result.show(10, truncate=False)

+-----+-------------------------------------+-------------------------------+
|rowid|str_value                            |cleaned                        |
+-----+-------------------------------------+-------------------------------+
|1    |Hello, World!!!                      |HELLO WORLD                    |
|2    |Python@#$%^&*()                      |PYTHON                         |
|3    |Привет, мир!!! 123                   |ПРИВЕТ МИР 123                 |
|4    |NULL                                 |NULL                           |
|5    |код (130) Номер 244=-55-56")         |КОД 130 НОМЕР 2445556          |
|6    |## Ключевые особенности 'SCALAR UDF:'|КЛЮЧЕВЫЕ ОСОБЕННОСТИ SCALAR UDF|
|7    |**Векто(р)и'\зованные опе%%%рации**  |ВЕКТОРИЗОВАННЫЕ ОПЕРАЦИИ       |
|8    |?*;'';&()_                           |                               |
|9    |Simple String №1                     |SIMPLE STRING 1                |
|10   |Simple String №2                     |SIMPLE STRING 2    

#### **Сравнение обработки строк с кэшированием и без**  
Допустим есть какая-то операция с текстом, длительностью 0.1 сек, и текстовое поле с повторяющимися значениями 

In [52]:
# SCALAR_ITER с кэшированием (эффективно)
@pandas_udf("string")
def cached_text_processing(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Обработчик текста с кэшированием"""
    cache = {}  # Кэш живет между всеми батчами
    
    def process_text(text):
        if text in cache:
            return cache[text]
            
        # Дорогая обработка
        time.sleep(.1)
        result = text.upper() if text else None
        cache[text] = result
        return result
    
    for batch in iterator:
        processed = batch.apply(process_text)
        yield processed

# SCALAR без кэширования (неэффективно для повторяющихся данных)
@pandas_udf("string")
def uncached_text_processing(series: pd.Series) -> pd.Series:
    """Обработчик текста без кэширования"""   
    # Кэша нет
    def process_text(text):
        time.sleep(.1)
        return text.upper() if text else None
    
    return series.apply(process_text)     

**DataSet**

In [53]:
import random

def generate_data(words=[], n=5000):
    """Генератор n-значений из списка words"""   
    data = []
    for i in range(n):
        text = random.choice(words)
        data.append((i + 1, text))
    return data
    
# Список категорий
words = ["apache", "spark", "hive", "oozie", "jupyter", "hue"]
# Генерация данных - n строк
data = generate_data(words, 500)

# Определение схемы
schema = StructType([
    StructField("rowid", IntegerType(), True),
    StructField("text", StringType(), True)
])

# Создание DataFrame
dfCacheTest = spark.createDataFrame(data, schema).repartition(2)
print(f'count: {dfCacheTest.count()}')
dfCacheTest.groupBy("text").count().show()

count: 500
+-------+-----+
|   text|count|
+-------+-----+
|jupyter|   82|
| apache|   89|
|  spark|   85|
|    hue|   86|
|  oozie|   68|
|   hive|   90|
+-------+-----+



In [54]:
# Настройки
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "50")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [55]:
print("Размер батча Arrow:", spark.conf.get("spark.sql.execution.arrow.maxRecordsPerBatch"))
print("Количество partitions:", dfCacheTest.rdd.getNumPartitions())
print("Размеры partitions:", dfCacheTest.rdd.glom().map(len).collect())

Размер батча Arrow: 50
Количество partitions: 2
Размеры partitions: [250, 250]


In [56]:
%%time
result = dfCacheTest.select("rowid", uncached_text_processing(col("text")).alias("text"))
ret = result.collect()

[Stage 89:>                                                         (0 + 2) / 2]

CPU times: user 11.4 ms, sys: 1.98 ms, total: 13.3 ms
Wall time: 25.2 s


In [57]:
%%time
result = dfCacheTest.select("rowid", cached_text_processing(col("text")).alias("text"))
ret = result.collect()

CPU times: user 11.2 ms, sys: 1.87 ms, total: 13.1 ms
Wall time: 713 ms


### 3. Обработка с внешними ресурсами
Некоторый ресурс загружается/рассчитывается один раз для каждой паритиции и пременяется к батчам расчета
 - создается тестовая модель и сохраняется в фвйл
 - загружается модель и применяется (к батчам)

In [58]:
import pickle
import os

#### **Пример батчевого применения модели с передачей 3х фичей-колонок через параметр**

In [59]:
# Cоздаем и сохраняем модель
def create_and_save_model():
    """Создает и сохраняет модель в файл"""

    model_weights = {
            'feature1_weight': 2.4,
            'feature2_weight': 1.2,
            'feature3_weight': 0.7,
            'bias': 0.4
        }
    # Сохраняем в pickle файл
    with open('/tmp/model_weights.pkl', 'wb') as f:
        pickle.dump(model_weights, f)
    
    print("Модель сохранена в /tmp/model_weights.pkl")

# Создаем модель
create_and_save_model()

Модель сохранена в /tmp/model_weights.pkl


In [60]:
@pandas_udf("double")
def ml_prediction_batched(iterator: Iterator[tuple[pd.Series, pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    """Применяет ML модель к батчам данных с 3 фичами"""
    
    # Инициализация модели один раз для всех батчей
    model_path = '/tmp/model_weights.pkl'
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            model_weights = pickle.load(f)
        print(f"Модель загружена из {model_path}: {model_weights}")
    else:
        # Fallback модель: веса для 3 фичей + bias
        model_weights = {
            'feature1_weight': 0.0,
            'feature2_weight': 0.0,
            'feature3_weight': 0.0,
            'bias': 0.0
        }
        print("Файл модели не найден, используется fallback модель")
    
    for feature1_batch, feature2_batch, feature3_batch in iterator:
        # Применяем линейную модель: w1*f1 + w2*f2 + w3*f3 + bias
        predictions = (
            feature1_batch * model_weights['feature1_weight'] + 
            feature2_batch * model_weights['feature2_weight'] + 
            feature3_batch * model_weights['feature3_weight'] + 
            model_weights['bias']
        )
        yield predictions.round(5)

In [61]:
# Создаем тестовые данные
test_df = spark.range(100).toDF("id") \
    .withColumn("feature1", rand() * 10) \
    .withColumn("feature2", rand() * 5) \
    .withColumn("feature3", rand() * 3)


In [62]:
test_df.dtypes

[('id', 'bigint'),
 ('feature1', 'double'),
 ('feature2', 'double'),
 ('feature3', 'double')]

In [63]:
# Применяем UDF
result_df = test_df.withColumn(
    "prediction", 
    ml_prediction_batched(col("feature1"), col("feature2"), col("feature3"))
)
result_df.show(10)

+---+------------------+------------------+------------------+----------+
| id|          feature1|          feature2|          feature3|prediction|
+---+------------------+------------------+------------------+----------+
|  0|3.9232266873712893|1.3831624950487542|0.8473818464765851|  12.06871|
|  1| 1.879715341767808| 4.793458188392268|1.4133132776998298|  11.65279|
|  2|  9.96196576952044|3.7851142788807723|0.7644882033571369|    29.386|
|  3| 7.661686053597718| 3.038556668302106|0.9796526608645141|  23.12007|
|  4| 1.101099703869317|2.7244348337705286|  2.07428815201402|   7.76396|
|  5|1.0976073284409449| 4.521899926604037|1.8267422973815974|   9.73926|
|  6| 7.564336171391773|2.7890481227500272|1.4162589368737033|  22.89265|
|  7| 8.683496072226472|1.8356516256531052| 1.767602527649105|  24.68049|
|  8|1.0376579659324392| 2.729749257998471|0.8856184230999437|   6.78601|
|  9| 6.249566348215032|1.7044574239431172|1.1813544432570073|  18.27126|
+---+------------------+--------------

Модель загружена из /tmp/model_weights.pkl: {'feature1_weight': 2.4, 'feature2_weight': 1.2, 'feature3_weight': 0.7, 'bias': 0.4}


#### **Пример батчевого применения модели с использованием DataFrame внутри UDF**

In [64]:
# Cоздаем и сохраняем модель
def create_and_save_model():
    """Создает и сохраняет модель в файл"""

    model_weights = {'weights': [2.4, 1.2, 0.7, 0.4], 'bias': 0.4} 
    # Сохраняем в pickle файл
    with open('/tmp/model_weights.pkl', 'wb') as f:
        pickle.dump(model_weights, f)
    
    print("Модель сохранена в /tmp/model_weights.pkl")

# Создаем модель
create_and_save_model()

Модель сохранена в /tmp/model_weights.pkl


In [65]:
@pandas_udf("double")
def ml_prediction_batched(iterator: Iterator[tuple[pd.Series, pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    """Применяет ML модель к батчам данных используя pandas DataFrame"""
    
    # Инициализация модели
    model_path = '/tmp/model_weights.pkl'
    if os.path.exists(model_path):
        with open(model_path, 'rb') as f:
            model_config = pickle.load(f)
        weights = model_config['weights']
        bias = model_config['bias']
        print(f"Модель загружена: weights={weights}, bias={bias}")
    else:
        weights = [0.6, 0.3, 0.1]
        bias = 0.5
        print("Используется fallback модель")
    
    for feature1_batch, feature2_batch, feature3_batch in iterator:
        # Создаем DataFrame из батчей
        batch_df = pd.DataFrame({
            'f1': feature1_batch,
            'f2': feature2_batch, 
            'f3': feature3_batch
        })
        
        # Применяем модель
        predictions = (
            batch_df['f1'] * weights[0] + 
            batch_df['f2'] * weights[1] + 
            batch_df['f3'] * weights[2] + 
            bias
        )
        
        yield predictions.round(5)

In [66]:
# Применяем UDF
result_df = test_df.withColumn(
    "prediction", 
    ml_prediction_batched(col("feature1"), col("feature2"), col("feature3"))
)
result_df.show(10)

+---+------------------+------------------+------------------+----------+
| id|          feature1|          feature2|          feature3|prediction|
+---+------------------+------------------+------------------+----------+
|  0|3.9232266873712893|1.3831624950487542|0.8473818464765851|  12.06871|
|  1| 1.879715341767808| 4.793458188392268|1.4133132776998298|  11.65279|
|  2|  9.96196576952044|3.7851142788807723|0.7644882033571369|    29.386|
|  3| 7.661686053597718| 3.038556668302106|0.9796526608645141|  23.12007|
|  4| 1.101099703869317|2.7244348337705286|  2.07428815201402|   7.76396|
|  5|1.0976073284409449| 4.521899926604037|1.8267422973815974|   9.73926|
|  6| 7.564336171391773|2.7890481227500272|1.4162589368737033|  22.89265|
|  7| 8.683496072226472|1.8356516256531052| 1.767602527649105|  24.68049|
|  8|1.0376579659324392| 2.729749257998471|0.8856184230999437|   6.78601|
|  9| 6.249566348215032|1.7044574239431172|1.1813544432570073|  18.27126|
+---+------------------+--------------

Модель загружена: weights=[2.4, 1.2, 0.7, 0.4], bias=0.4


#### **Пример применения иодели с обработкой ошибок**

In [67]:
# Cоздаем и сохраняем модель
def create_and_save_model():
    """Создает и сохраняет модель в файл"""

    model_weights = [2.4, 1.2, 0.7, 0.4] 
    # Сохраняем в pickle файл
    with open('/tmp/model_weights.pkl', 'wb') as f:
        pickle.dump(model_weights, f)
    
    print("Модель сохранена в /tmp/model_weights.pkl")

# Создаем модель
create_and_save_model()

Модель сохранена в /tmp/model_weights.pkl


In [68]:
@pandas_udf("double")
def ml_prediction_batched_safe(iterator: Iterator[tuple[pd.Series, pd.Series, pd.Series]]) -> Iterator[pd.Series]:
    """Версия с обработкой ошибок"""
    
    try:
        # Загрузка модели
        model_path = '/tmp/model_weights.pkl'
        if os.path.exists(model_path):
            with open(model_path, 'rb') as f:
                model_weights = pickle.load(f)
        else:
            model_weights = [0.33, 0.33, 0.34, 0.0]  # равные веса + нулевой bias
            
    except Exception as e:
        print(f"Ошибка загрузки модели: {e}")
        model_weights = [1.0, 0.0, 0.0, 0.0]  # fallback к первой фиче
    
    for feature1_batch, feature2_batch, feature3_batch in iterator:
        try:
            # Проверка на NaNи замена на 0
            f1 = feature1_batch.fillna(0.0)
            f2 = feature2_batch.fillna(0.0)
            f3 = feature3_batch.fillna(0.0)
            
            # Применение модели
            predictions = (
                f1 * model_weights[0] + 
                f2 * model_weights[1] + 
                f3 * model_weights[2] + 
                model_weights[3]
            )
            
            # Ограничиваем предсказания разумными пределами
            predictions = predictions.clip(-100, 100)
            
            yield predictions.round(5)
            
        except Exception as e:
            print(f"Ошибка в батче: {e}")
            # Возвращаем нули в случае ошибки
            yield pd.Series([0.0] * len(feature1_batch))

In [69]:
# Применяем UDF
result_df = test_df.withColumn(
    "prediction", 
    ml_prediction_batched_safe(col("feature1"), col("feature2"), col("feature3"))
)
result_df.show(10)

+---+------------------+------------------+------------------+----------+
| id|          feature1|          feature2|          feature3|prediction|
+---+------------------+------------------+------------------+----------+
|  0|3.9232266873712893|1.3831624950487542|0.8473818464765851|  12.06871|
|  1| 1.879715341767808| 4.793458188392268|1.4133132776998298|  11.65279|
|  2|  9.96196576952044|3.7851142788807723|0.7644882033571369|    29.386|
|  3| 7.661686053597718| 3.038556668302106|0.9796526608645141|  23.12007|
|  4| 1.101099703869317|2.7244348337705286|  2.07428815201402|   7.76396|
|  5|1.0976073284409449| 4.521899926604037|1.8267422973815974|   9.73926|
|  6| 7.564336171391773|2.7890481227500272|1.4162589368737033|  22.89265|
|  7| 8.683496072226472|1.8356516256531052| 1.767602527649105|  24.68049|
|  8|1.0376579659324392| 2.729749257998471|0.8856184230999437|   6.78601|
|  9| 6.249566348215032|1.7044574239431172|1.1813544432570073|  18.27126|
+---+------------------+--------------

### 4. Обработка с несколькими колонками через lambda
Простой пример обработки нескольких параметров

In [70]:
# Данные с двумя колонками
data = [(i, float(i), float(i * 0.1)) for i in range(1, 101)]
dfTwoCols = spark.createDataFrame(data, ["rowid", "value", "weight"])

In [71]:
@pandas_udf("double")
def complex_calculation_iter(
    iterator: Iterator[tuple[pd.Series, pd.Series]]
) -> Iterator[pd.Series]:
    """Сложные вычисления с несколькими входными колонками"""
    
    for value_batch, weight_batch in iterator:
        # Взвешенное преобразование
        result = (value_batch * weight_batch).apply(lambda x: x ** 0.5 if x > 0 else 0)
        yield result



In [72]:
result = dfTwoCols.select(
    "rowid", 
    complex_calculation_iter(col("value"), col("weight")).alias("weighted_result")
)
result.show(10)

+-----+-------------------+
|rowid|    weighted_result|
+-----+-------------------+
|    1|0.31622776601683794|
|    2| 0.6324555320336759|
|    3| 0.9486832980505139|
|    4| 1.2649110640673518|
|    5| 1.5811388300841898|
|    6| 1.8973665961010278|
|    7| 2.2135943621178655|
|    8| 2.5298221281347035|
|    9| 2.8460498941515415|
|   10| 3.1622776601683795|
+-----+-------------------+
only showing top 10 rows



### 5. Обработка временных рядов
Сложный пример обработки со скользящим окном

In [73]:
# Временные ряды данных
ts_data = [(i, float(10 + 5 * np.sin(i * 0.1) + np.random.randn())) 
           for i in range(1, 201)]
df_time_series = spark.createDataFrame(ts_data, ["rowid", "value"])

In [74]:
import numpy as np

@pandas_udf("double")
def sliding_window_stats(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    """Вычисляет статистики скользящего окна между батчами"""
    
    window_buffer = []
    window_size = 5
    
    for batch in iterator:
        results = []
        
        for value in batch:
            window_buffer.append(value)
            
            # Поддерживаем размер окна
            if len(window_buffer) > window_size:
                window_buffer.pop(0)
            
            # Вычисляем среднее по окну
            window_mean = np.mean(window_buffer)
            results.append(window_mean)
        
        yield pd.Series(results)

In [75]:
result = df_time_series.select("rowid", "value", sliding_window_stats(col("value")).alias("smoothed"))
result.show(15)

+-----+------------------+------------------+
|rowid|             value|          smoothed|
+-----+------------------+------------------+
|    1|  9.59114300771293|  9.59114300771293|
|    2| 9.581042952640015| 9.586092980176472|
|    3|12.943249802228252|10.705145254193733|
|    4|11.721315411056718| 10.95918779340948|
|    5| 12.46465589770894|11.260281414269372|
|    6| 11.39846418076172| 11.62174564887913|
|    7|12.676705711663274|12.240878200683781|
|    8| 13.69770304420748|12.391768849079627|
|    9|12.765640970715113|12.600633961011305|
|   10|14.583052942385153| 13.02431336994655|
|   11|13.855398110388373|13.515700155871878|
|   12|14.368501680042856|13.854059349547793|
|   13| 14.21608431485657|13.957735603677614|
|   14| 16.77952683445124|14.760512776424838|
|   15|14.973977708282339|14.838697729604274|
+-----+------------------+------------------+
only showing top 15 rows



### 6. Сравнение производительности SCALAR UDF и SCALAR_ITER UDF

In [76]:
import time

# Обычный SCALAR UDF
@pandas_udf("double")
def scalar_square(series: pd.Series) -> pd.Series:
    return series ** 2

# SCALAR_ITER UDF
@pandas_udf("double")
def scalar_iter_square(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for batch in iterator:
        yield batch ** 2

# Большой датасет
large_data = [(i, float(i)) for i in range(1, 200001)]
large_df = spark.createDataFrame(large_data, ["id", "value"])

# Тестирование SCALAR
start = time.time()
result1 = large_df.select("id", scalar_square(col("value")).alias("squared"))
count1 = result1.count()
scalar_time = time.time() - start

# Тестирование SCALAR_ITER
start = time.time()
result2 = large_df.select("id", scalar_iter_square(col("value")).alias("squared"))
count2 = result2.count()
scalar_iter_time = time.time() - start

print(f"SCALAR UDF time: {scalar_time:.2f} seconds")
print(f"SCALAR_ITER UDF time: {scalar_iter_time:.2f} seconds")

SCALAR UDF time: 0.12 seconds
SCALAR_ITER UDF time: 0.12 seconds


### 5. Обработка ошибок в SCALAR и SCALAR_ITER

In [77]:
# SCALAR - ошибка влияет на всю partition
@pandas_udf("double")
def scalar_errorprocessing(series: pd.Series) -> pd.Series:
    # Если ошибка - вся partition падает
    return series.apply(lambda x: 1/x if x != 0 else float('inf'))

# SCALARITER - можно обработать ошибки по батчам
@pandas_udf("double")
def scalar_iter_errorprocessing(iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for batchnum, batch in enumerate(iterator):
        try:
            result = batch.apply(lambda x: 1/x if x != 0 else float('inf'))
            yield result
        except Exception as e:
            print(f"Error in batch {batch_num}: {e}")
            # Возвращаем безопасные значения
            yield pd.Series([0.0] * len(batch))

## SCALAR_ITER используется если:

1. **Большие датасеты** - если нужно контролировать использование памяти
2. **Stateful обработка** -  если нужно сохранять состояние между батчами
3. **Инициализация ресурсов** -  если дорого создавать объекты для каждого вызова
4. **Кэширование** - если можно переиспользовать вычисления
5. **Потоковая обработка** - если данные обрабатываются как поток

⚠️ **SCALAR** может вызвать OutOfMemory на больших данных  

✅ **SCALAR_ITER** обработает по частям

## Сравнительная таблица:
| Характеристика | SCALAR | SCALAR_ITER |
|---------------|---------|-------------|
| **Память** | Загружает всю partition | Обрабатывает батчами |
| **Состояние** | Не сохраняется | Сохраняется между батчами |
| **Инициализация** | При каждом вызове | Один раз на partition |
| **Производительность** | Быстрее для малых данных | Лучше для больших данных |
| **Сложность** | Проще в реализации | Требует понимания итераторов |
| **Кэширование** | Ограниченное | Эффективное |

In [78]:
spark.stop()